<a href="https://colab.research.google.com/github/jrhulteen/hello-world/blob/master/Notebook2Hulteen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 2

Name: Jacob Hulteen

In [153]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [154]:
#import the data:

#make sure the path on the line below corresponds to the path where you put your dataset.
data = pd.read_csv('/content/drive/MyDrive/CS167/vehicles.csv')
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,...,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,1850,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,...,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3500,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,...,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-10250,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1450,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,...,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,0,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3500,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,...,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-10250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2550,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,...,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-5500,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42098,14.982273,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1750,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,...,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,24.0000,0.0,37.0000,0.0,Compact Cars,1993,-1500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
42099,14.330870,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,386.391304,23,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1700,0,Regular,Regular Gasoline,-1,-1,28,0.0,0,0.0,0.0,0.0,...,9996,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,25.0000,0.0,39.0000,0.0,Compact Cars,1993,-1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

## Exercise #1: 
Work with the right subset: You're not going to work with the whole data set, just the cars with "Regular" listed in the "fuelType" column. So, to start, make a subset of the original data.

In [170]:
regular = data[data['fuelType']=='Regular']
regular1 = regular[['comb08', 'year', 'cylinders','displ']]
regular1

,comb08,year,cylinders,displ
0,21,1985,4.0,2.0
1,11,1985,12.0,4.9
2,27,1985,4.0,2.2
3,11,1985,8.0,5.2
5,22,1993,4.0,1.8
...,...,...,...,...
42097,25,1993,4.0,1.9
42098,22,1993,4.0,2.2
42099,23,1993,4.0,2.2
42100,21,1993,4.0,2.2


## Exercise #2:

Check for null values in the target and predictor columns. If you have a null target value, you will need to throw that example out. If you have a null predictor value, you can either fill them in with something (like the mean/median) or you can drop those rows from the data set. Describe in a markup cell what you decided to do with the null data.

In [181]:
#Replacing the empty values in cylinders and displ with the mean values
avg_cylinders = regular1['cylinders'].mean()
avg_displ = regular1['displ'].mean()
data1 = regular1.copy()
data1['cylinders'] = regular1['cylinders'].fillna(avg_cylinders)
data1['displ'] = regular1['displ'].fillna(avg_displ)
data1.isna().any()

comb08       False
year         False
cylinders    False
displ        False
dtype: bool

*Took the empty cells and replaced them with the average of the rest of the column*

## Exercise #3:

Write up a k-nearest-neighbors function like the one you made for the iris data set in class. It should be able to make mpg ("comb08") predictions for new cars based on the year, cylinders, and displacement. You should also be able to specify what you want to use as k.

In [182]:
import numpy
def knn(predictor, data1, k):
   #calculate the distance
  data1['distance_to_new'] = numpy.sqrt(
    (predictor['year']-data1['year'])**2
    +(predictor['cylinders']-data1['cylinders'])**2
    +(predictor['displ']-data1['displ'])**2)
  #sort by 'distance'
  sorted_data = data1.sort_values(['distance_to_new']) 
  #return the mean() of the k closest neighbors
  return sorted_data.iloc[0:k]['comb08'].mean()

## Exercise #4:

Demonstrate that your function works by making up some new values for hypothetical cars and using your function to display the predicted mpg ("comb08") for that car.

In [183]:
testcar = {}
testcar['year']=1990
testcar['cylinders']=4.0
testcar['displ']=2.0
#Running function
knn(testcar, data1,3)

19.666666666666668

## Exercise #5:

Make a copy of the data and normalize the training data using Z-score. Predict the mpg of a particular car using your k-nearest-neighbors function with both the normalized and non-normalized training data. Compare your results. Use a markup cell to describe and explain the differences in a few sentences.

In [185]:
data2 = data1.copy()
#Calculating mean
year_mean = data2['year'].mean()
cylinders_mean = data2['cylinders'].mean()
displ_mean = data2['displ'].mean()
#Calculating std
year_std = data2['year'].std()
cylinders_std = data2['cylinders'].std()
displ_std = data2['displ'].std()
#Replacing columns with z-score
data2['year'] = (data2['year']-year_mean)/year_std
data2['cylinders'] = (data2['cylinders']-cylinders_mean)/cylinders_std
data2['displ'] = (data2['displ']-displ_mean)/displ_std
#Normalize testcar
testcar['year'] = (1990-year_mean)/year_std
testcar['cylinders'] = (4.0-cylinders_mean)/cylinders_std
testcar['displ'] = (2.0-displ_mean)/displ_std
knn(testcar, data2, 3)

20.666666666666668

For the non-normalized data, using the parameters of year=1990, cylinders=4.0, and displ=2.0, I got 19.67 mpg using 3 nearest neighbors. Using the same parameters but normalizing the data, I got 20.67 mpg using 3 nearest neighbors.  The non-normalized data used the actual data entries of year, cylinders, and displacement which wasn't horribly wrong for this dataset due to the fact they're all numeric to start with, but year was a relatively different numeric value than cylinders and displacement.  As for the normalized data where I calculated the z-score for year, cylinders, and displacement, the values were weighted more similar to one another, causing the mpg to change from 19.67 (non-normalized) to 20.67 (normalized).